<a href="https://colab.research.google.com/github/aditj/IRS-Paper/blob/master/IRS_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
import math

## Helper Functions

We can sample the Channel coefficients from a rician distribution with parameters $\beta$ and $K$. $\mathbf{h_d}$ is constant and $\mathbf{h_s}$ is sampled form a circularly symmetric standard Complex Gaussian  
$$\mathbf{h} = \sqrt{\frac{\beta K}{1+K}} \mathbf{h_d} + \sqrt{\frac{\beta }{1+K}} \mathbf{h_s}$$

In [48]:
# get the channel coefficients from the rician distribution
def rician(beta,k,h_d=0 ,num_carriers=50,num_channels=1):
    """
    returns a (num_carriers x num_channels) matrix with complex numbers whose magnitude would follow a rician distribution 
    beta is a distance dependent path loss parameter 
    K is ratio of deterministic to scattered power
    """
    det = math.sqrt(beta*k/(1+k))* h_d
    ran = math.sqrt(beta/((1+k)*2)) * np.random.randn(num_carriers, num_channels*2).view(np.complex128)
    return ran+det

## System Parameters

In [47]:
# General parameters

M = 2 #Num of discrete phase-shifts
N_x = 16
N_y = 16
N_e = N_x * N_y #Number of IRS elements

# Channel information
#direct channel
beta_d = 1
k_d = 1
const_d = 0
#BS to IRS channel
beta_bi = 1
k_bi = 1
const_bi = 0
#IRS to User channel
beta_iu = 1
k_iu = 1
const_iu = 0

## Declaring variables

In [50]:
## CSI

H_d = rician(beta=beta_d, k=k_d)
H_bi = rician(beta=beta_bi,k=k_bi,num_channels=N_e)
H_iu = rician(beta=beta_iu,k=k_iu,num_channels=N_e)

## Theta

IRS_config = np.exp(2j*np.random.randint(M,size=N_e)*np.pi/M)
Direct = np.ones(1).astype(np.complex128)
Theta = np.concatenate((Direct,IRS_config)).reshape((N_e+1,1))


## Final channel

H_id = np.multiply(H_bi,H_iu)
H = np.concatenate((H_d,H_id), axis=1)
G = np.matmul(H,Theta)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=4b319ffc-f2ce-4490-9e28-3a7eb21abad0' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>